In [ ]:
import os, uuid

import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas

from holonote.annotate import Annotator

What you see in this notebook will depend on whether you've run this notebook before and written annotations to the `annotations.db` database! For reproducibility, the rest of the notebook will assume the `annotations.db` has been deleted (if it exists):

# Load some example data

This basic example will be of a timeseries where we want to annotate various time intervals to illustrate the basics of the annotation system. Note that annotators can annotate all sorts of elements (e.g. `Image`, `Scatter` etc.) with many different region types which will be demonstrated later.

In [ ]:
xvals  = np.linspace(-4, 0, 202)
yvals  = np.linspace(4, 0, 202)
xs, ys = np.meshgrid(xvals, yvals)

alpha, beta = 1,0
image = hv.Image(np.sin(((ys/alpha)**alpha+beta)*xs), kdims=['A', 'B']).opts(cmap='greens')
image

In the simplest case, simply wrap the element (here in a curve) in an `Annotator`:

In [ ]:
annotator = Annotator(image, fields=['description'], region_types=['Range'])

The `fields` argument lists the fields associated with the annotations we will be defining. When working with tabular data (the typical case), you can thing of fields as the columns of your table containing information about annotated regions.

Here we supplied an element to annotator to the `Annotator` but note that most of the functionality of annotators can be made available by specifying the key dimensions and their types. The following is largely equivalent to the above declaration:

```python
annotator = Annotator({'A': np.float64, 'B': np.float64}, fields=['description'])
```

Now we can create an overlay of our element, a dynamicmap that shows the defined annotation regions and a dynamicmap used to define new regions:

In [ ]:
annotator.overlay()

The default output of `.overlay` is designed to be useful by default. Options on controlling the displayed layers and their styles can be found in the **Using `.overlay` section**.

**Note** The tools made available by the region editor is appropriate to the dimensionality of the element (here, a single key dimension on the x-axis).

Also note that if we had used the key dimension type specification in the constructor instead of the element, the following would be equivalent:

```python
annotator.overlay(element=speed_curve)
```

## Basic operations on annotations

#### Adding single annotations

Using the box select tool, you can define a region of interest to annotate and run the following cell:

In [ ]:
annotator.add_annotation(description='My first annotation!')

You can set the range of interest programmatically as well:

In [ ]:
annotator.set_range(-0.25,0.25,-0.25,0.25)
annotator.add_annotation(description='A programmatically defined annotation')

You should now see that annotated regions have appeared in the plot above. We can view a `DataFrame` of the data collected as follows:

It is important to note the automatically generated `uuid` index (by default) which will be discussed in the next section

To persist these annotations, we call the `.commit()` method:

In [ ]:
annotator.commit()

Now if we restart the notebook session (without forgetting to set `PERSIST=True` at the start!), you will see your annotations are automatically loaded and displayed.

#### Simple selection of annotations

The `uuid` index column of the dataframe above is how we refer to individual annotations. We may use this column directly, for instance we could get the uuid of the last annotation directly as follows:

In [ ]:
uuid_of_last_annotation = annotator.df.index[-1]
f'Last UUID in the dataframe: {uuid_of_last_annotation}'

Note that uuid values are randomly generated (by default) which means we do not know what these values will be ahead of time. As a result we need a programmatic way to access them. Using the dataframe index directly is awkward, so annotators offer a more natural, interactive way to select annotations - simply click on them in the plot to select them.

Click on a range region in the plot above and run the following cell to see it's uuid:

In [ ]:
annotator.selected_index # None if no annotations are selected

See the 'Advanced selections' section for more information on selection.

#### Deleting single annotations

Now we have added some annotations and have a way to select them, we can delete them.


Select an annotation on the plot and run the following cell to delete it:

In [ ]:
selected_index = annotator.selected_index if annotator.selected_index else annotator.df.index[-1]
annotator.delete_annotation(selected_index)

#### Updating annotations

First let us add a new annotation to update:

In [ ]:
annotator.set_range(-0.35,-0.15,-0.35,-0.25)
annotator.add_annotation(description='An annotation description we will update...')

Now click on the new annotation and run the following cell:

In [ ]:
annotator.update_annotation_fields(annotator.selected_index if annotator.selected_index else annotator.df.index[-1], 
                                   description='The description is now updated!')

To verify this operation worked, note how the hover information has updated in the plot above.

Remember that all your changes to the annotations are not persisted to the database until you call `commit`! Frequent commits are recommended.

In [ ]:
annotator.commit()

## Explicit primary keys

We have the *option* to specify our own UUID when creating an annotation if necessary:

In [ ]:
input_uuid = 'deadcafe'
description = f'Annotation with set UUID {input_uuid!r}'
annotator.set_range(0.35,0.15,0.35,0.25)
annotator.add_annotation(description=description, uuid=input_uuid)
description

This option is offered to give you full control over what is entered into the database, however it is recommended that you do *not* specify the primary key value yourself unless you really need to due to the following caveats:

#### *Caveats when picking your own primary keys*

While it may occasionally be convenient to name your own annotations in notebooks with set primary key values, you should be aware that the primary key value you pick is then supplied to the database.  This implies the following restrictions:

* It is your responsibility to ensure the key is unique and not used by any other annotation in the database.
* It is your responsibility to ensure the key is of the valid type and format for storage in the database.

For these reasons, it is generally recommended you allow the annotator to pick the key values automatically (a process you can customize as detailed in the Persisting Annotations notebook) and then refer to annotations via the dataframe index or interactive selection as previously demonstrated.

Now we have demonstrated the creation of an explicitly named annotation, we can delete it and revert to using the recommended mechanisms for selecting annotations:

In [ ]:
annotator.delete_annotation('deadcafe') # List more uuids to delete multiple annotations at once

In [ ]:
annotator.commit()

## Adding and deleting multiple annotations

#### Loading from a dataframe

Sometimes we have a dataframe with some annotation we want to load and it does not make sense to loop over the rows for insertion. Suppose we have the following dataframe:

In [ ]:
startx, endx = [-0.1,-0.2,-0.3], [0.1,0.2,0.3]
starty, endy = [-0.2,-0.3,-0.4], [0.2,0.3,0.4]
descriptions = ['Annotation %d' % d for d in range(3)]
data = pd.DataFrame({'startx':startx, 'endx':endx, 'starty':starty, 'endy':endy, 'description':descriptions})
data

To load this data, we *first* need to define the data in the annotation fields (for this annotator this is the `description` field):

In [ ]:
annotator.define_fields(data[['description']])

Now we can define the starts and ends of our annotation regions as follows:

In [ ]:
annotator.define_ranges(data['startx'], data['endx'], data['starty'], data['endy'])

The `DataFrame`/`Series` index (here the default pandas index) is what links our ranges to their corresponding field entries.

Note the following rules:

1. The index values supplied to the `define_` region methods must first be defined by `define_fields`.
2. Before we can call the `.commit` method, every index supplied to `define_fields` must have been given at least one region.

As with `add_annotation`, the annotator automatically create uuid primary key values and the original index is ignored:

In [ ]:
annotator.commit()

This can be overridden by supplying `preserve_index=True` to the `define_fields` method as now described.

### Preserving the index

Sometimes the annotations you are loading have meaningful primary keys defined elsewhere (e.g. some other pre-existing database) that need to be preserved. This is possible by supplying `preserve_index=True` to the `define_fields` method.

*Note: The user bears the same responsibilities for using appropriate index values as described in the  ***Caveats when picking your own primary keys*** section!*


In [ ]:
uuids = ['DEADC0DE', 'CAFED00D', 'BAADF00D']
indexed_data = pd.DataFrame({'uuid':uuids, 
                             'startx':startx, 'endx':endx, 'starty':starty, 'endy':endy, 
                             'description':[f'Labelled {el}' for el in descriptions]}).set_index('uuid')
indexed_data

In [ ]:
annotator.define_fields(indexed_data[['description']], preserve_index=True)

In [ ]:
annotator.define_ranges(indexed_data['startx'], indexed_data['endx'], indexed_data['starty'], indexed_data['endy'])

In [ ]:
annotator.df

In [ ]:
annotator.commit()

## Selecting and highlighting annotations

Earlier we styled the indicators with `color='red', alpha=0.2`. To highlight a select a specific indicator, we can create a dimension expression to assign selected and non-selected indicators different values. Here we have a highlighter that uses a value of `0.6` for selected indicators and `0.1` for non-selected indicators. We can then apply these values to the `alpha` option:

In [ ]:
highlighter = annotator.selected_dim_expr(0.6, 0.1)
annotator.element * annotator.indicators().opts(color='red', alpha=highlighter)  * annotator.region_editor()

Now you can use the `Tap` tool to directly select indicators.

You can now delesect by collecting `select_by_index()` without any arguments:

In [ ]:
annotator.select_by_index()

And we can select one or more indicators by index value:

In [ ]:
annotator.df

In [ ]:
annotator.select_by_index(annotator.df.index[0])

You can access the selected indicators using the `selected_indices` parameter (watchable):

In [ ]:
annotator.selected_indices # parameter

This demonstrates the basics without storing additional metadata, using the undo system or persisting any data.

Note that everything shown so far can be achieved with simple Python API and a single line of display code:

```
annotator.element * annotator.indicators().opts(color='red', alpha=annotator.selected_dim_expr(0.6, 0.1)) * annotator.editable()
```


## Using `.overlay`

The `.overlay` method is a shortcut to building the following overlay:

```
annotator.element * annotator.indicators() * annotator.region_editor()
```

Where each layer can be enabled/disabled and styled individually: the above is equivalent to `annotator.overlay(element=True, indicators=True, editor=True)`.


Note, if either building the overlay yourself or using `.overlay` with `element=False` you can use `annotator.element` *or* you can use the original element after adding the necessary tools with ```speed_curve.opts(tools=annotator.edit_tools)```.

### Styling the annotator

You can set the style either through the `_style` keywords in `.overlay`:

In [ ]:
annotator.overlay(range_style={'color': 'yellow', 'alpha': 0.4}, 
                  edit_range_style={'alpha': 0.4, 'line_alpha': 1, 'color':'blue'})

Or once at the class-level as follows:

In [ ]:
annotator.indicator.range_style['color']= 'green'
annotator.indicator.edit_range_style['color'] = 'yellow'